In [ ]:
!pip install keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.applications.xception import (Xception, preprocess_input, decode_predictions)
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
datagen = ImageDataGenerator(
       rotation_range=40,
       width_shift_range=0.2,
       height_shift_range=0.2,
       rescale=1./255,
       shear_range=0.2,
       zoom_range=0.2,
       horizontal_flip=True,
       fill_mode='nearest')

In [ ]:
save_to_dir=r'C:\dataML\train'
img = load_img('dataML/train/glaucoma/01_g.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the preview/ directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='dataML', save_prefix='glaucoma', save_format='jpeg'):
   i += 1
   if i > 20:
       break  # otherwise the generator would loop indefinitely

# __Set up Model__


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

# __Train the Model__

In [ ]:
batch_size = 16
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
       rescale=1./255,
       shear_range=0.2,
       zoom_range=0.2,
       horizontal_flip=True)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
       'dataML/train',  # this is the target directory
       target_size=(150, 150),  # all images will be resized to 150x150
       batch_size=batch_size,
       class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
       'dataML/validation',
       target_size=(150, 150),
       batch_size=batch_size,
       class_mode='binary')



# __Fit the Model__

In [ ]:
model.fit_generator(
       train_generator,
       steps_per_epoch=2000 // batch_size,
       epochs=5,
       validation_data=validation_generator,
       validation_steps=800 // batch_size)
model.save_weights('first_try.h5')



# __Pre-processing__

In [ ]:
image_size = (150, 150)
im = keras.preprocessing.image.load_img("filepath.jpg",
target_size=image_size,
grayscale=False)
def prepare_image(img):
   img = img_to_array(img)
   img = np.expand_dims(img, axis=0)
   img = preprocess_input(img)
   # return the processed image
   return img
image = prepare_image(im)
image.shape